In [1]:
import functools

import clingo

from typing import Sequence

In [2]:
def solve(programs: Sequence[str], grounding_context=None, sep:str=' ', report_answersets=True):
    ctl = clingo.Control(("--models", "0"))
    for program in programs:
        ctl.add("base", (), program)

    ctl.ground((("base", ()),), grounding_context)

    models = []

    with ctl.solve(yield_=True) as solve_handle:
        for i, model in enumerate(solve_handle):
            assert isinstance(model, clingo.Model)
            symbols = model.symbols(shown=True)
            if report_answersets:
                print("Answer {}: {}{}{}{}{}".format(i + 1, "{", sep, sep.join(map(str, sorted(symbols))), sep, "}"))
            models.append(symbols)
        solve_result = solve_handle.get()
        cardinality_suffix = ""
        if not solve_result.exhausted:
            cardinality_suffix = "+"
        print(solve_result, "{}{}".format(len(models), cardinality_suffix))

    return models
run = functools.partial(solve, report_answersets=False)

In [3]:
def check(programs: Sequence[str], consistency_checks:str, grounding_context=None):
    print("Run without checks:")
    wo_checks = solve(programs, grounding_context, report_answersets=False)
    print("Run with checks:")
    w_checks = solve((*programs, consistency_checks), grounding_context, report_answersets=False)
    return len(wo_checks) == len(w_checks)

In [4]:
resources = """

resource("Geld").

"""
run([resources]);

SAT 1


In [5]:
accounts = """

account("Lieferant:Geld").
account("Empfänger:Geld").

"""
run([accounts]);

SAT 1


In [6]:
agents = """

agent("Lieferant").
agent("Empfänger").

"""
run([agents]);

SAT 1


In [7]:
gridworld = """

width(3).
height(4).

coordinates((1..W,1..H)) :- width(W), height(H).

adjacent(P1, P2) :-
  coordinates(P1), coordinates(P2),
  P1 = (X1, Y1),
  P2 = (X2, Y2),
  |X1 - X2| + |Y1 - Y2| = 1.

diagonal(P1, P2) :-
  coordinates(P1), coordinates(P2),
  P1 = (X1, Y1),
  P2 = (X2, Y2),
  |X1 - X2| = 1,
  |Y1 - Y2| = 1.

surrounding(P1, P2) :-
  diagonal(P1, P2).
surrounding(P1, P2) :-
  adjacent(P1, P2).

"""
solve([gridworld]);

Answer 1: { coordinates((1,1)) coordinates((1,2)) coordinates((1,3)) coordinates((1,4)) coordinates((2,1)) coordinates((2,2)) coordinates((2,3)) coordinates((2,4)) coordinates((3,1)) coordinates((3,2)) coordinates((3,3)) coordinates((3,4)) height(4) width(3) adjacent((1,1),(1,2)) adjacent((1,1),(2,1)) adjacent((1,2),(1,1)) adjacent((1,2),(1,3)) adjacent((1,2),(2,2)) adjacent((1,3),(1,2)) adjacent((1,3),(1,4)) adjacent((1,3),(2,3)) adjacent((1,4),(1,3)) adjacent((1,4),(2,4)) adjacent((2,1),(1,1)) adjacent((2,1),(2,2)) adjacent((2,1),(3,1)) adjacent((2,2),(1,2)) adjacent((2,2),(2,1)) adjacent((2,2),(2,3)) adjacent((2,2),(3,2)) adjacent((2,3),(1,3)) adjacent((2,3),(2,2)) adjacent((2,3),(2,4)) adjacent((2,3),(3,3)) adjacent((2,4),(1,4)) adjacent((2,4),(2,3)) adjacent((2,4),(3,4)) adjacent((3,1),(2,1)) adjacent((3,1),(3,2)) adjacent((3,2),(2,2)) adjacent((3,2),(3,1)) adjacent((3,2),(3,3)) adjacent((3,3),(2,3)) adjacent((3,3),(3,2)) adjacent((3,3),(3,4)) adjacent((3,4),(2,4)) adjacent((3,4),

In [8]:
objectTypes = """

objectType(pipeline).
objectType(valve).

"""
run([objectTypes]);

SAT 1


In [9]:
actions = """

"""
run([actions]);

SAT 1


In [10]:
items = """

item("Lieferung").
item("Bezahlung").

"""
run([items]);

SAT 1


In [11]:
attainment = """

attainment(object_change(object(pipeline, sink), change(liquidLevel, full))).
attainment(object_event(object(account, "Lieferant:Geld"), event(transaction, (3, "Geld")))).

"""
run([attainment]);

SAT 1


In [12]:
balance_rules = """

occurs(object(account, Account), change(balance, OldBalance + Change), T) :-
  account(Account), time(T),
  observe(object(account, Account), value(balance, OldBalance), T),
  Change = Income + Expense + Withdrawn + Deposit,
  Change != 0,
  Income = #sum { N,Res : occurs(object(account, Account), event(income, (N, Res)), T) },
  Expense = #sum { -N,Res : occurs(object(account, Account), event(expense, (N, Res)), T) },
  Withdrawn = #sum { -N,Res,Because,Into : occurs(object(account, Account), event(transaction, ((N, Res), Account, Into, Because)), T), account(Into) },
  Deposit = #sum { N,Res,Because,From : occurs(object(account, Account), event(transaction, ((N, Res), From, Account, Because)), T), account(From) }.

"""
run([balance_rules]);

SAT 1


<block>:4:3-19: info: atom does not occur in any rule head:
  account(Account)

<block>:4:21-28: info: atom does not occur in any rule head:
  time(T)

<block>:5:3-67: info: atom does not occur in any rule head:
  observe(object(account,Account),value(balance,OldBalance),T)

<block>:8:27-87: info: atom does not occur in any rule head:
  occurs(object(account,Account),event(income,(N,Res)),T)

<block>:9:29-90: info: atom does not occur in any rule head:
  occurs(object(account,Account),event(expense,(N,Res)),T)

<block>:10:44-135: info: atom does not occur in any rule head:
  occurs(object(account,Account),event(transaction,((N,Res),Account,Into,Because)),T)

<block>:10:137-150: info: atom does not occur in any rule head:
  account(Into)

<block>:11:41-132: info: atom does not occur in any rule head:
  occurs(object(account,Account),event(transaction,((N,Res),From,Account,Because)),T)

<block>:11:134-147: info: atom does not occur in any rule head:
  account(From)



In [13]:
static_physics = """

occurs(object(pipeline, B), change(liquidLevel, empty), T) :-
  occurs(object(pipeline, B), change(flows, object(pipeline, C)), T),
  observe(object(pipeline, A), value(connected, object(pipeline, B)), T),
  not occurs(object(pipeline, A), change(flows, object(pipeline, B)), T).

occurs(object(pipeline, B), change(liquidLevel, empty), T) :-
  occurs(object(pipeline, B), change(flows, object(pipeline, C)), T),
  not observe(object(pipeline, _), value(connected, object(pipeline, B)), T).

occurs(object(pipeline, C), change(liquidLevel, full), T) :-
  occurs(object(pipeline, B), change(flows, object(pipeline, C)), T).

occurs(object(pipeline, A), change(flows, ()), T) :-
  not observe(object(pipeline, A), value(flows, ()), T),
  observe(object(pipeline, A), value(liquidLevel, empty), T).

occurs(object(pipeline, A), change(flows, object(pipeline, B)), T) :-
  observe(object(pipeline, A), value(flows, ()), T),
  observe(object(pipeline, A), value(connected, object(pipeline, B)), T),
  observe(object(pipeline, A), value(liquidLevel, full), T).


"""
run([static_physics]);

SAT 1


<block>:5:3-73: info: atom does not occur in any rule head:
  observe(object(pipeline,A),value(connected,object(pipeline,B)),T)

<block>:10:7-77: info: atom does not occur in any rule head:
  observe(object(pipeline,#P0),value(connected,object(pipeline,#X1)),#X2)

<block>:16:7-56: info: atom does not occur in any rule head:
  observe(object(pipeline,A),value(flows,()),T)

<block>:17:3-61: info: atom does not occur in any rule head:
  observe(object(pipeline,A),value(liquidLevel,empty),T)

<block>:20:3-52: info: atom does not occur in any rule head:
  observe(object(pipeline,A),value(flows,()),T)

<block>:21:3-73: info: atom does not occur in any rule head:
  observe(object(pipeline,A),value(connected,object(pipeline,B)),T)

<block>:22:3-60: info: atom does not occur in any rule head:
  observe(object(pipeline,A),value(liquidLevel,full),T)



In [14]:
action_consequences = """

occurs(object(account, From), event(transaction, ((Amount, Resource), From, Into, Because)), T) :-
  occurs(Provider, action(give, ((Amount, Resource), From, Into, Because)), T),
  occurs(Beneficiary, action(accept, ((Amount, Resource), From, Into, Because)), T).

occurs(object(account, Into), event(transaction, ((Amount, Resource), From, Into, Because)), T) :-
  occurs(Provider, action(give, ((Amount, Resource), From, Into, Because)), T),
  occurs(Beneficiary, action(accept, ((Amount, Resource), From, Into, Because)), T).

occurs(object(pipeline, A), change(liquidLevel, full), T) :-
  observe(Agent, value(at, Coordinates), T),
  observe(object(pipeline, A), value(at, Coordinates), T),
  occurs(Agent, action(fill, object(pipeline, A)), T).

"""
run([action_consequences]);

SAT 1


<block>:4:3-79: info: atom does not occur in any rule head:
  occurs(Provider,action(give,((Amount,Resource),From,Into,Because)),T)

<block>:5:3-84: info: atom does not occur in any rule head:
  occurs(Beneficiary,action(accept,((Amount,Resource),From,Into,Because)),T)

<block>:8:3-79: info: atom does not occur in any rule head:
  occurs(Provider,action(give,((Amount,Resource),From,Into,Because)),T)

<block>:9:3-84: info: atom does not occur in any rule head:
  occurs(Beneficiary,action(accept,((Amount,Resource),From,Into,Because)),T)

<block>:12:3-44: info: atom does not occur in any rule head:
  observe(Agent,value(at,Coordinates),T)

<block>:13:3-58: info: atom does not occur in any rule head:
  observe(object(pipeline,A),value(at,Coordinates),T)

<block>:14:3-54: info: atom does not occur in any rule head:
  occurs(Agent,action(fill,object(pipeline,A)),T)



In [15]:
static = """

observe(Who, What, ST) :-
  init(Who, What),
  time(ST), not time(ST-1).

observe(Who, value(Property, Value), T+1) :-
  time(T+1),
  observe(Who, value(Property, Value), T),
  not occurs(Who, change(Property, _), T).

observe(Who, value(Property, NewValue), T+1) :-
  time(T+1),
  occurs(Who, change(Property, NewValue), T).

"""
run([static]);

SAT 1


<block>:4:3-18: info: atom does not occur in any rule head:
  init(Who,What)

<block>:5:3-11: info: atom does not occur in any rule head:
  time(ST)

<block>:5:17-27: info: atom does not occur in any rule head:
  time((ST+-1))

<block>:8:3-12: info: atom does not occur in any rule head:
  time((T+1))

<block>:10:7-42: info: atom does not occur in any rule head:
  occurs(#X0,change(#X1,#P2),#X3)

<block>:13:3-12: info: atom does not occur in any rule head:
  time((T+1))

<block>:14:3-45: info: atom does not occur in any rule head:
  occurs(Who,change(Property,NewValue),T)



In [16]:
contract_domain = """

contract("Wasser").

component("Lieferung").
component("Bezahlung").

contract_component("Wasser", "Lieferung").
contract_component("Wasser", "Bezahlung").

component_item("Lieferung", "Lieferung").
component_item("Bezahlung", "Bezahlung").

item_attainment("Lieferung", object_change(object(pipeline, sink), change(liquidLevel, full))).
item_attainment("Bezahlung", object_event(object(account, "Lieferant:Geld"), event(transaction, (3, "Geld")))).

component_provider("Lieferung", "Lieferant").
component_beneficiary("Lieferung", "Empfänger").

component_provider("Bezahlung", "Empfänger").
component_beneficiary("Bezahlung", "Lieferant").

component_deadline("Lieferung", 4).
component_deadline("Bezahlung", 5).

component("Lieferung Nachfrist").

component_deadline("Lieferung Nachfrist", 7).

component_conversion("Lieferung", "Lieferung Nachfrist").

"""
run([contract_domain]);

SAT 1


In [17]:
contract_static = """

componentState("Inaktiv").
componentState("Initialisiert").
componentState("Gewährleistung").
componentState("Verzug (Schuldner)").
componentState("Inaktiv [GLE]").
componentState("Initialisiert [GLE]").
componentState("Gewährleistung [GLE]").
componentState("Verzug (Schuldner) [GLE]").

state_dualState("Inaktiv", "Inaktiv [GLE]").
state_dualState("Initialisiert", "Initialisiert [GLE]").
state_dualState("Gewährleistung", "Gewährleistung [GLE]").
state_dualState("Verzug (Schuldner)", "Verzug (Schuldner) [GLE]").

transitionAction("A").
transitionAction("GLE").
transitionAction("LE").
transitionAction("V(S)").

state_transAction_state("Inaktiv", "A", "Initialisiert").
state_transAction_state("Initialisiert","LE","Gewährleistung").
state_transAction_state("Initialisiert","V(S)","Verzug (Schuldner)").

state_transAction_state(Dual, TA, DualNext) :-
  state_dualState(State, Dual),
  state_dualState(Next, DualNext),
  state_transAction_state(State, TA, Next).

state_transAction_state(State, "GLE", DualState) :- state_dualState(State, DualState).

state_transAction_state(State, "GLE", NextDualState) :-
  state_transAction_state(State, TA, NextState),
  state_dualState(NextState, NextDualState).

state_transAction_state(State, TA, NextDualState) :-
  state_transAction_state(State, TA, NextState),
  state_dualState(NextState, NextDualState).

component_item(Conversion, Item) :-
  component_conversion(Component, Conversion),
  component_item(Component, Item).

component_provider(Conversion, Provider) :-
  component_conversion(Component, Conversion),
  component_provider(Component, Provider).

component_beneficiary(Conversion, Provider) :-
  component_conversion(Component, Conversion),
  component_beneficiary(Component, Provider).

"""
solve([contract_static], sep='\n');

Answer 1: {
componentState("Gewährleistung")
componentState("Gewährleistung [GLE]")
componentState("Inaktiv")
componentState("Inaktiv [GLE]")
componentState("Initialisiert")
componentState("Initialisiert [GLE]")
componentState("Verzug (Schuldner)")
componentState("Verzug (Schuldner) [GLE]")
transitionAction("A")
transitionAction("GLE")
transitionAction("LE")
transitionAction("V(S)")
state_dualState("Gewährleistung","Gewährleistung [GLE]")
state_dualState("Inaktiv","Inaktiv [GLE]")
state_dualState("Initialisiert","Initialisiert [GLE]")
state_dualState("Verzug (Schuldner)","Verzug (Schuldner) [GLE]")
state_transAction_state("Gewährleistung","GLE","Gewährleistung [GLE]")
state_transAction_state("Inaktiv","A","Initialisiert")
state_transAction_state("Inaktiv","A","Initialisiert [GLE]")
state_transAction_state("Inaktiv","GLE","Inaktiv [GLE]")
state_transAction_state("Inaktiv","GLE","Initialisiert [GLE]")
state_transAction_state("Inaktiv [GLE]","A","Initialisiert [GLE]")
state_transAction_st

<block>:42:3-46: info: atom does not occur in any rule head:
  component_conversion(Component,Conversion)

<block>:46:3-46: info: atom does not occur in any rule head:
  component_conversion(Component,Conversion)

<block>:50:3-46: info: atom does not occur in any rule head:
  component_conversion(Component,Conversion)



In [18]:
contract_dynamic = """

component_dualComponent(Component, Dual) :-
 component(Component), component(Dual),
 contract(Contract),
 Component != Dual,
 contract_component(Contract, Component), contract_component(Contract, Dual).

% If the transition event does not occur then there is no transition
deontic(object(component, Component), value(impossible, SuccState), T+1) :-
  component(Component), componentState(SuccState), time(T+1),
  componentState(State), transitionAction(TransAction),
  deontic(object(component, Component), value(state, State), T),
  state_transAction_state(State, TransAction, SuccState),
  not deontic(object(component, Component), event(TransAction), T).

% If two states have the same successor state and both states can be reached they will not transition
deontic(object(component, Component), value(impossible, State1), T+1) :-
  component(Component), componentState(State1), time(T+1),
  componentState(State), componentState(State2), transitionAction(TransAction1), transitionAction(TransAction2),
  deontic(object(component, Component), value(state, State), T),
  state_transAction_state(State, TransAction1, State1),
  state_transAction_state(State, TransAction2, State2),
  state_transAction_state(State1, TransAction2, SuccState),
  state_transAction_state(State2, TransAction1, SuccState),
  deontic(object(component, Component), event(TransAction1), T),
  deontic(object(component, Component), event(TransAction2), T).

% If a transition is not impossible then it will be executed
deontic(object(component, Component), change(state, SuccState), T) :-
  component(Component), componentState(SuccState), time(T),
  componentState(State), time(T+1),
  deontic(object(component, Component), value(state, State), T),
  state_transAction_state(State, _, SuccState),
  not deontic(object(component, Component), value(impossible, SuccState), T+1).

% INERTIA
deontic(object(component, Component), value(state, State), T+1) :-
  component(Component), componentState(State), time(T+1),
  time(T),
  deontic(object(component, Component), value(state, State), T),
  not deontic(object(component, Component), change(state, _), T).

% TRANSITION
deontic(object(component, Component), value(state, SuccState), T+1) :-
  component(Component), componentState(SuccState), time(T+1),
  time(T), componentState(State),
  deontic(object(component, Component), value(state, State), T),
  deontic(object(component, Component), change(state, SuccState), T).

%%%
deontic(object(component, Component), event("LE"), T) :-
  component(Component), time(T),
  item(Item), attainment(Attainment),
  component_item(Component, Item),
  item_attainment(Item, Attainment),
  Attainment = object_event(Object, Event),
  Event = event(transaction, (Amount, Resource)),
  occurs(Object, event(transaction, ((Amount, Resource), _, _ , Component)), T).

deontic(object(component, Component), event("LE"), T) :-
  component(Component), time(T),
  item(Item), attainment(Attainment), agent(Provider), agent(Beneficiary),
  component_provider(Component, Provider),
  component_beneficiary(Component, Beneficiary),
  component_item(Component, Item),
  item_attainment(Item, Attainment),
  Attainment = object_event(Object, Event),
  occurs(Object, Event, T).

deontic(object(component, Component), event("LE"), T) :-
  component(Component), time(T),
  item(Item), attainment(Attainment),
  component_item(Component, Item),
  item_attainment(Item, Attainment),
  Attainment = object_value(Object, Value),
  observe(Object, Value, T).

deontic(object(component, Component), event("LE"), T) :-
  component(Component), time(T),
  item(Item), attainment(Attainment),
  component_item(Component, Item),
  item_attainment(Item, Attainment),
  Attainment = object_change(Object, Change),
  occurs(Object, Change, T).

deontic(object(component, Component), event("GLE"),T) :-
  component(Component), time(T),
  component(Dual),
  component_dualComponent(Component, Dual),
  deontic(object(component, Dual), event("LE"), T).

deontic(object(component, Conversion), event("GLE"),T) :-
  component(Conversion), time(T),
  component_conversion(Component, Conversion),
  deontic(object(component, Component), event("GLE"),T).

deontic(object(component, Component), event("V(S)"),T-1) :-
  component_deadline(Component, T),
  state_transAction_state(State, "V(S)", _),
  not deontic(object(component, Component), event("LE"), T-1),
  deontic(object(component, Component), value(state, State), T-1).

deontic(object(component, Conversion), event("A"), T) :-
  component_conversion(Component, Conversion),
  deontic(object(component, Component), event("V(S)"), T).

"""
run([contract_dynamic]);

SAT 1


<block>:4:2-22: info: atom does not occur in any rule head:
  component(Component)

<block>:4:24-39: info: atom does not occur in any rule head:
  component(Dual)

<block>:5:2-20: info: atom does not occur in any rule head:
  contract(Contract)

<block>:7:2-41: info: atom does not occur in any rule head:
  contract_component(Contract,Component)

<block>:7:43-77: info: atom does not occur in any rule head:
  contract_component(Contract,Dual)

<block>:11:3-23: info: atom does not occur in any rule head:
  component(Component)

<block>:11:25-50: info: atom does not occur in any rule head:
  componentState(SuccState)

<block>:11:52-61: info: atom does not occur in any rule head:
  time((T+1))

<block>:12:3-24: info: atom does not occur in any rule head:
  componentState(State)

<block>:12:26-55: info: atom does not occur in any rule head:
  transitionAction(TransAction)

<block>:14:3-57: info: atom does not occur in any rule head:
  state_transAction_state(State,TransAction,SuccState)

<bl

In [19]:
instance = """

time(1..6).
init(object(pipeline, source), value(at, (2,3))).
init(object(pipeline, source), value(liquidLevel, empty)).
init(object(pipeline, source), value(connected, object(pipeline, connect))).

init(object(pipeline, sink), value(at, (2,1))).
init(object(pipeline, sink), value(liquidLevel, empty)).

init(object(pipeline, connect), value(at, (2,2))).
init(object(pipeline, connect), value(liquidLevel, empty)).
init(object(pipeline, connect), value(connected, object(pipeline, sink))).

init(agent("Lieferant"), value(at, (2,3))).
init(agent("Empfänger"), value(at, (2,1))).

init(object(account, "Lieferant:Geld"), value(balance, 0)).
init(object(account, "Empfänger:Geld"), value(balance, 3)).

deontic(object(component, "Bezahlung"), value(state, "Initialisiert"), 1).
deontic(object(component, "Lieferung"), value(state, "Initialisiert"), 1).
deontic(object(component, "Lieferung Nachfrist"), value(state, "Inaktiv"), 1).

occurs(agent("Lieferant"), action(fill, object(pipeline, source)), 2).

occurs(agent("Empfänger"), action(give, ((3, "Geld"), "Empfänger:Geld", "Lieferant:Geld", "Bezahlung")), 4).
occurs(agent("Lieferant"), action(accept, ((3, "Geld"), "Empfänger:Geld", "Lieferant:Geld", "Bezahlung")), 4).

"""
run([instance]);

SAT 1


In [20]:
prg = (
    resources,
    accounts,
    agents,
    gridworld,
    objectTypes,
    actions,
    items,
    attainment,
    balance_rules,
    static_physics,
    static,
    contract_domain,
    contract_static,
    contract_dynamic,
    action_consequences,
    instance
)

In [21]:
solve((*prg,
       '#show init/2.',
       '#show occurs/3.',
       '#show observe/3.'
       '#show deontic/3.'), sep='\n');

Answer 1: {
init(agent("Empfänger"),value(at,(2,1)))
init(agent("Lieferant"),value(at,(2,3)))
init(object(account,"Empfänger:Geld"),value(balance,3))
init(object(account,"Lieferant:Geld"),value(balance,0))
init(object(pipeline,connect),value(at,(2,2)))
init(object(pipeline,connect),value(connected,object(pipeline,sink)))
init(object(pipeline,connect),value(liquidLevel,empty))
init(object(pipeline,sink),value(at,(2,1)))
init(object(pipeline,sink),value(liquidLevel,empty))
init(object(pipeline,source),value(at,(2,3)))
init(object(pipeline,source),value(connected,object(pipeline,connect)))
init(object(pipeline,source),value(liquidLevel,empty))
deontic(object(component,"Bezahlung"),event("GLE"),4)
deontic(object(component,"Bezahlung"),event("LE"),4)
deontic(object(component,"Bezahlung"),change(state,"Gewährleistung [GLE]"),4)
deontic(object(component,"Bezahlung"),value(impossible,"Gewährleistung"),2)
deontic(object(component,"Bezahlung"),value(impossible,"Gewährleistung"),3)
deontic(object